<a href="https://colab.research.google.com/github/LauraJouLee/Fall2021-TermProject-DigitalHumanities/blob/main/Step_2_Spotify_Audio_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spotipy --upgrade

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from __future__ import print_function
import spotipy
import spotipy.util as util
import sys
#from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

#### You have to create a DASHBOARD to get the required CLIENT_ID and Client_SECRET code
#### Spotify Developer Dashboard Link: https://developer.spotify.com/dashboard/

# Authorization Code Flow

### To support the Authorization Code Flow Spotipy provides a utility method util.prompt_for_user_token that will attempt to authorize the user. 


```
scope = 'user-read-private'
token = util.prompt_for_user_token('lauraleetaiwan', scope, 
client_id='b9ab0fc630594519a6cbde985f51d242',client_secret='59a63526f72e43f5a46b0a06e104ce5e',redirect_uri='http://localhost:8080/')
sp = spotipy.Spotify(auth=token)
```




# Client Credentials Flow
### Client credentials flow is appropriate for requests that do not require access to a user’s private data. Even if you are only making calls that do not require authorization, using this flow yields the benefit of a higher rate limit



```
client_credentials_manager = SpotifyClientCredentials(client_id='b9ab0fc630594519a6cbde985f51d242',client_secret='59a63526f72e43f5a46b0a06e104ce5e')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
```

In [ ]:
## set Authorization and get token

client_credentials_manager = SpotifyClientCredentials(client_id='b9ab0fc630594519a6cbde985f51d242',client_secret='59a63526f72e43f5a46b0a06e104ce5e')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
##Clean data from Billboard
Billboard_Hot100 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DH_Final_Hot100.csv')

artist_list=[]
song_list=[]

a_toBeReplace = ['Featuring', ' X ', ' x ', ' & ', ' + ', ', ']
s_toBeReplace = ['(', ':', '.', ', ']

for artist in Billboard_Hot100['artist']:
    artist_list.append(artist)

for elem in a_toBeReplace :
    # Check if string is in the main string
    for k,v in enumerate(artist_list):
        if elem in v:
            # Replace the string
            artist_list[k] = v.replace(elem, ' ')

for song in Billboard_Hot100['song']:
    song_list.append(song)

for elem in s_toBeReplace :
    # Check if string is in the main string
    for k,v in enumerate(song_list):
        if elem in v:
            # Replace the string
            song_list[k] = v.replace(elem, ' ')



# Get Spotify Track ID

### Output
1. artist, track, track_id
2. error list: no track_id output >> add manually


In [ ]:
track_id_list=[]
error_artist=[]
error_track=[]


#search from list
for i , j in enumerate(artist_list):
    for m ,n in enumerate(song_list):
        if i== m:
            search_output = sp.search(q='artist:' + j + ' track:' + n, type='track')
            
            try :
                search_output = pd.DataFrame(search_output['tracks'])
                search_output = search_output['items'][0]                
                track_id_list.append(search_output.get('id'))
                
            except:
                error_artist.append(j)
                error_track.append(n)


In [ ]:
#view error outputs with no duplicate values

for (a,b) in zip(error_artist,error_track):
    pd.DataFrame({'error_artist':error_artist,'error_track': error_track}).to_csv('Spotify_error_list.csv')


In [ ]:
##get audio features
artist_name_list=[]
artist_genres_list=[]
track_list=[]
audio_feature_list=[]

for track_id in track_id_list:
    artist_name_list.append(sp.track(track_id)['album']['artists'][0]['name'])
    artist_genres_list.append(sp.artist(sp.track(track_id)['album']['artists'][0]['id'])['genres'])
    track_list.append(sp.track(track_id)['name'])
    audio_feature_list.append(sp.audio_features(track_id)[0])



In [ ]:
track_info = pd.DataFrame({
    #'artist_id' : artist_id_list,
    'artist_name' : artist_name_list, 
    'artist_genres' : artist_genres_list,
    #'album_id' : album_id_list, 
    #'album_name': album_name_list, 
    'track_list': track_list})


track_info = track_info.loc[track_info.astype(str).index]
audio_feature = pd.DataFrame(audio_feature_list)  

track_info.join(audio_feature).to_csv('Spotify_nonerror_track_merged_2.csv')


In [ ]:
##check error
len(error_list)